# 第4课：通话记录摘要器

在本课中，我们将为一个常见的客户用例——摘要——编写一个复杂的提示。具体来说，我们将总结冗长的客户服务通话记录。我们的目标是为客户支持指标总结客户服务通话。我们希望获得完整的客户服务通话摘要，以评估我们客户支持团队的效率。这意味着我们将排除那些存在连接问题、语言障碍以及其他阻碍有效摘要的问题的通话。

假设我们为Acme公司工作，这是一家销售智能家居设备的公司。该公司每天处理数百个客户服务电话，需要一种方法来快速将这些对话转化为**有用、结构化的数据**。

一些重要的考虑因素包括：
*   通话可能简短而顺利，也可能冗长而复杂。
*   客户来电可能涉及从简单的Wi-Fi连接问题到复杂的系统故障等任何事情。
*   我们需要特定格式的摘要，以便日后易于分析。
*   我们必须小心，不要在摘要中包含任何客户个人信息。

为了帮助我们，我们将遵循之前描述的最佳实践：
*   使用系统提示来设定场景。
*   优化提示结构以获得最佳性能。
*   给出清晰的指令并定义你期望的输出。
*   使用XML标签来组织信息。
*   处理特殊情况和边缘场景。
*   提供示例来引导模型。

---

## 理解数据

现在我们已经理解了任务，接下来让我们看看将要处理的数据。在本课中，我们将使用来自Acme公司智能家居设备支持团队的各种模拟客户服务通话记录。这些记录将帮助我们创建一个能够处理不同场景的健壮提示。

让我们检查一下我们可能会遇到的几种通话记录类型：

一个简短而简单的文本记录：

In [ ]:
call1 = """
客服代表：感谢您致电Acme智能家居支持。我是亚历克斯。有什么可以帮您的吗？
顾客：您好，我的智能灯泡打不开了。
客服代表：我明白了。您尝试过重置灯泡了吗？
顾客：哦，没有。我该怎么做？
客服代表：只需断电5秒，然后再通电。它应该就会重置。
顾客：好的，我试试看。谢谢！
客服代表：不客气。如果需要进一步帮助，请再次致电我们。
"""

一个中等长度的、最终有解决方案的文本记录：

In [ ]:
call2 = """
客服代表：Acme智能家居支持，我是杰米。今天有什么可以帮您的吗？
顾客：嗨，杰米，我的Acme智能恒温器（SmartTherm）无法保持我设定的温度。它设定的是72度，但房子里要热得多。
客服代表：很抱歉听到这个问题。我们来排查一下故障。您的智能恒温器连接到Wi-Fi了吗？
顾客：是的，显示屏上显示着Wi-Fi符号。
客服代表：好的。我们来重新校准您的智能恒温器。请按住菜单按钮5秒钟。
顾客：好的，按了。出现了一个新菜单。
客服代表：太好了。导航到“校准”（Calibration）并按下选择键。调整温度以匹配您房间温度计的读数。
顾客：好的，我已经调到79度来匹配了。
客服代表：很好。按下选择键确认。它将进行重新校准，这可能需要几分钟。一小时后回来检查一下是否已修复。
顾客：好的，我会照做的。谢谢你的帮助，杰米。
客服代表：不客气！今天还有其他我可以帮助您的吗？
顾客：没有了，就这些。再次感谢。
客服代表：感谢您选择Acme智能家居。祝您有愉快的一天！
"""

一个更长的、没有解决方案的通话记录：

In [ ]:
call3 = """
客服代表：感谢您联系Acme智能家居支持。我是莎拉。今天有什么可以帮您的吗？
顾客：嗨，莎拉，我的Acme SecureHome系统出了点问题。警报总是随机地响起来。
客服代表：很抱歉听到这个问题。您能告诉我这是什么时候开始发生的吗？
顾客：大约两天前开始的。现在已经响了三次了，总是在半夜。
客服代表：我明白了。发生这种情况时，控制面板上有没有显示任何错误信息？
顾客：没有，我没注意到。但我每次都迷迷糊糊的。
客服代表：明白了。我们来检查几件事。首先，您能确认所有的门窗都关好了吗？
顾客：是的，我都检查过了。它们都好好的。
客服代表：好的。接下来，我们检查一下您控制面板的电池。您能告诉我低电量指示灯亮了吗？
顾客：等一下……没有，电池指示灯看起来正常。
客服代表：好的。可能是您的某个传感器出了故障。我想运行一个诊断程序，但这需要将您转接到我们的技术团队。可以吗？
顾客：好的，没问题。我只想把这个问题解决掉。这真的很扰民。
客服代表：我完全理解。我现在就给您转接。他们将能够运行完整的系统诊断，并希望能为您解决问题。
顾客：好的，谢谢。
客服代表：不客气。感谢您的耐心等待，希望您今天过得愉快。
"""

这些示例展示了我们需要处理的各种通话类型和考量因素：
*   通话长度差异巨大。
*   通话涉及各种支持问题（简单修复、设备故障、复杂问题）。
*   有些通话以问题解决告终，有些则仍是未解决的案例。
*   有些通话需要后续跟进。

在构建我们的提示时，我们需要确保它能够有效地总结所有这些类型的通话，提取关键信息并以一致、结构化的格式呈现。
在下一节中，我们将开始逐步构建我们的提示，以处理这种多样化的通话记录。

---

## 提示的简单版本
现在我们已经理解了我们的任务以及正在处理的数据类型，接下来我们开始构建我们的提示。我们将从一个基本版本开始，并逐步对其进行完善，以处理通话摘要任务的复杂性。

让我们从这个非常简单的提示开始，它概述了基本任务：

In [ ]:
prompt = """
总结以下客户服务通话记录。重点关注主要问题、问题如何解决以及任何需要后续跟进的事项。

{文本记录}
"""

这个基本提示让Claude大致了解了我们想要什么，但它有几个局限性：

*   它没有指定所需的输出格式，这可能导致摘要不一致。
*   它没有提供关于如何处理不同情况（例如未解决的问题或信息不足）的指导。
*   它没有对长度或内容设置任何限制，可能导致摘要过长或过于详细。
*   它没有指示Claude省略个人信息，这可能导致隐私问题。

话虽如此，让我们测试一下，看看它的表现如何：

In [ ]:
from anthropic import Anthropic
from dotenv import load_dotenv

load_dotenv()
client = Anthropic()

def summarize_call(transcript):
    final_prompt = prompt.format(transcript=transcript)
    # Make the API call
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        messages=[
            {"role": "user", "content": final_prompt}
        ]
    )
    print(response.content[0].text)

In [ ]:
summarize_call(call1)

```txt
以下是客户服务通话记录的摘要：

主要问题：
顾客无法打开他们的Acme智能灯泡。

解决方案：
客服代表指导顾客通过断电5秒后再通电的方式重置灯泡。这应该会重置灯泡并使其能够打开。

后续跟进：
客服代表告诉顾客，如果在尝试重置程序后仍然有问题，请再次致电。没有提及其他后续跟进。
```

In [ ]:
summarize_call(call2)

```txt
摘要：

主要问题：顾客的Acme智能恒温器（SmartTherm）无法保持设定的72华氏度温度，导致室内温度远高于设定值。

解决方案：客服代表指导顾客完成了智能恒温器的重新校准过程。这包括进入“校准”菜单，调整温度以匹配顾客房间温度计的读数（本例中为79华氏度），并确认新设置。重新校准过程可能需要几分钟才能完成。

需要后续跟进：客服建议顾客在一小时后检查恒温器，以确认重新校准过程完成后问题是否已解决。
```

In [ ]:
summarize_call(call3)

```txt
以下是客户服务通话记录的摘要：

主要问题：
顾客的Acme SecureHome警报系统出现问题，即使所有门窗都已正确关闭，警报仍会在半夜随机响起。

问题如何解决：
客服代表首先让顾客检查控制面板上是否有任何错误信息，并确认电池电量未低。当这些基本故障排除步骤未能发现问题时，客服代表判断可能是某个传感器出现故障，因此需要将顾客转接到技术支持团队进行全面的系统诊断。

需要后续跟进：
技术支持团队需要对顾客的SecureHome系统运行诊断程序，以查明是哪个传感器可能导致了误报，然后修复或更换这些组件。一旦诊断完成并执行了修复/更换，应再次联系顾客，以确保随机警报问题已得到解决。
```

正如您所看到的，尽管Claude确实提供了摘要，但其格式并不便于系统性分析。摘要可能过长或过短，并且可能无法始终如一地涵盖我们感兴趣的所有要点。

在接下来的步骤中，我们将开始为我们的提示添加更多结构和指导，以解决这些局限性。我们将看到每一次添加如何提高Claude摘要的质量和一致性。

请记住，提示工程是一个迭代过程。我们从简单开始，然后逐步完善我们的提示。

---

## 添加系统提示

最简单的入手点是使用一个系统提示，它为Claude设定了整体上下文和角色，有助于在整个交互过程中引导其行为。

让我们从这个系统提示开始：

In [ ]:
system = """
您是一位专业的客户服务分析师，擅长从通话记录中提取关键信息并以结构化格式进行总结。
您的任务是分析客户服务通话记录，并生成简洁、准确的摘要，同时保持专业的语气。
"""

---

## 构建我们的主提示

接下来，我们将开始编写主提示。我们将依赖以下一些提示技巧：

*   将长文档（我们的文本记录）放在顶部。
*   添加详细的指令和输出格式要求。
*   引入XML标签来构建提示和输出。
*   给Claude留出“大声思考”的空间。

由于这个提示可能会变得相当长，我们将单独编写各个部分，然后再将它们组合起来。

### 输入数据
在使用像Claude这样的大型语言模型时，将长文档（比如我们的通话记录）放在提示的开头至关重要。这能确保Claude在接收具体指令之前，拥有所有必要的上下文。我们还应该使用XML标签来标识提示中的文本记录：

In [ ]:
prompt_pt1 = """
分析以下客户服务通话记录，并生成一份交互的JSON格式摘要：

<transcript>
[在这里插入通话文本记录]
</transcript>
"""

### 指令与输出格式

在我们进一步深入之前，让我们清晰地思考一个好的结构化输出格式应该是什么样子。为了在解析结果时更方便，通常最简单的方法是要求Claude返回JSON格式的响应。在这种情况下，一个好的JSON应该是什么样的呢？

至少，我们的JSON输出应该包含以下内容：
*   一个状态，表明Claude是否有足够的信息来生成摘要。我们稍后会回到这一点。目前，我们假设所有摘要的状态都是“COMPLETE”，这意味着Claude可以生成摘要。
*   客户问题的摘要
*   通话是否需要额外后续跟进
*   任何后续行动的详细信息（如果需要，例如回电给客户等）
*   问题是如何解决的
*   对话中模糊或不明确的点的列表

以下是一个建议的JSON结构示例：

```json
{
  "summary": {
    "customerIssue": "主要问题或致电原因的简要描述",
    "resolution": "问题如何处理或解决（如果适用）",
    "followUpRequired": true/false,
    "followUpDetails": "任何必要的后续行动的描述，如果不需要则为null"
  },
  "status": "COMPLETE",
  "ambiguities": ["对话中任何不清楚或模糊点的列表，如果没有则为空数组"]
}
```

让我们创建提示的新部分，其中包含具体指令，包括：
*   创建一个侧重于主要问题、解决方案和任何所需后续行动的摘要。
*   生成遵循我们特定、标准化格式的JSON输出。
*   在摘要中省略具体的客户信息。
*   使摘要的每个部分保持简短。

以下是提供输出指令的尝试，包括我们具体的输出JSON格式：

In [ ]:
prompt_pt2 = """
指令：
1. 仔细阅读通话记录。
2. 分析通话记录，重点关注主要问题、解决方案以及任何需要后续跟进的事项。
3. 根据指定结构生成一个JSON对象——总结交互的关键方面。

重要指南：
- 保密性：省略所有具体的客户数据，如姓名、电话号码和电子邮件地址。
- 字符限制：将每个文本字段限制在最多100个字符。
- 在您的摘要中保持专业的语气。

输出格式：
生成一个具有以下结构的JSON对象：
<json>
{
  "summary": {
    "customerIssue": "主要问题或致电原因的简要描述",
    "resolution": "问题如何处理或解决（如果适用）",
    "followUpRequired": true/false,
    "followUpDetails": "任何必要的后续行动的描述，如果不需要则为null"
  },
  "status": "COMPLETE",
  "ambiguities": ["对话中任何不清楚或模糊点的列表，如果没有则为空数组"]
}
</json>
"""

---

## 使用XML标签并给Claude思考空间

接下来，我们将采用另外两种提示策略：给Claude思考空间和使用XML标签。
*   我们将要求Claude首先输出包含其分析的`<thinking>`标签。
*   然后，我们将要求Claude将其JSON输出放在`<json>`标签内。

这是我们第一个提示草稿的最后一部分：

In [ ]:
prompt_pt3 = """
在生成JSON之前，请在`<thinking>`标签中分析通话记录。
请包含您对主要问题、解决方案、后续跟进要求以及任何模糊之处的识别。
然后，在`<json>`标签中提供您的JSON输出。
"""

通过要求Claude将其分析放在`<thinking>`标签内，我们是在促使它在形成最终JSON输出之前，先分解其思考过程。这鼓励了一种更彻底、更有条理的转录分析方法。
`<thinking>`部分允许我们（以及潜在的其他审阅者或系统）看到Claude的推理过程。这种透明度对于调试和质量保证目的至关重要。

通过将分析（`<thinking>`）与结构化输出（`<json>`）分开，我们清晰地区分了Claude对转录的解释和其格式化的摘要。这在某些情况下可能很有用，例如我们希望单独审查分析而不是JSON输出；同时，通过将JSON内容隔离在`<json>`标签内，我们使得解析最终响应并捕获我们想要处理的JSON变得容易。

---

## 测试我们更新后的提示

以下是提示的完整版本，通过结合我们目前编写的各个提示片段构建而成：

In [ ]:
system = """
您是一位专业的客户服务分析师，擅长从通话记录中提取关键信息并以结构化格式进行总结。
您的任务是分析客户服务通话记录，并生成简洁、准确的摘要，同时保持专业的语气。
"""

prompt = """
分析以下客户服务通话记录，并生成一份交互的JSON格式摘要：

<transcript>
[在此处插入通话记录]
</transcript>

指令：
1. 仔细阅读通话记录。
2. 分析通话记录，重点关注主要问题、解决方案以及任何需要后续跟进的事项。
3. 根据指定结构生成一个JSON对象，总结交互的关键方面。

重要指南：
- 保密性：省略所有具体的客户数据，如姓名、电话号码和电子邮件地址。
- 字符限制：将每个文本字段限制在最多100个字符。
- 在您的摘要中保持专业的语气。

输出格式：
生成一个具有以下结构的JSON对象：
<json>
{
  "summary": {
    "customerIssue": "主要问题或致电原因的简要描述",
    "resolution": "问题如何处理或解决（如果适用）",
    "followUpRequired": true/false,
    "followUpDetails": "任何必要的后续行动的描述，如果不需要则为null"
  },
  "status": "COMPLETE",
  "ambiguities": ["对话中任何不清楚或模糊点的列表，如果没有则为空数组"]
}
</json>

在生成JSON之前，请在<thinking>标签中分析通话记录。
请包含您对主要问题、解决方案、后续跟进要求以及任何模糊之处的识别。
然后，在<json>标签中提供您的JSON输出。
"""

以下是一个我们可以用来测试提示的函数：

In [ ]:
def summarize_call_with_improved_prompt(transcript):
    final_prompt = prompt.replace("[INSERT CALL TRANSCRIPT HERE]", transcript)
    # Make the API call
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        system=system,
        max_tokens=4096,
        messages=[
            {"role": "user", "content": final_prompt}
        ]
    )
    print(response.content[0].text)

让我们使用之前定义的一些通话记录来测试这个提示：

In [ ]:
summarize_call_with_improved_prompt(call1)

```txt
<thinking>
从通话记录来看，主要问题似乎是顾客无法打开他们的智能灯泡。客服代表提供的解决方案是，通过断电5秒后再通电的方式重置灯泡。

客服代表确实提出如果顾客需要进一步帮助可以回电，这表明如果重置未能解决问题，可能需要后续跟进。然而，没有提供具体的后续跟进细节。

对话中似乎没有明显的模糊之处。
</thinking>

<json>
{
  "summary": {
    "customerIssue": "无法打开智能灯泡",
    "resolution": "客服代表指导顾客通过断电5秒后再通电的方式重置灯泡",
    "followUpRequired": true,
    "followUpDetails": "建议顾客如果重置未能解决问题，请回电"
  },
  "status": "COMPLETE",
  "ambiguities": []
}
</json>
```

In [ ]:
summarize_call_with_improved_prompt(call2)

```txt
<thinking>
主要问题：顾客的Acme智能恒温器（SmartTherm）无法保持设定的72华氏度温度，导致室内温度远高于设定值。

解决方案：客服代表指导顾客完成了智能恒温器的重新校准过程，具体步骤如下：
1. 让顾客按住菜单按钮5秒钟。
2. 导航到“校准”菜单并选择它。
3. 调整温度以匹配顾客房间温度计的读数79华氏度。
4. 确认新的校准设置。

需要后续跟进：是的，客服代表指示顾客在一小时后回来检查，看重新校准是否解决了温度问题。

模糊之处：无
</thinking>

<json>
{
  "summary": {
    "customerIssue": "恒温器无法保持设定温度，导致室内温度高出许多。",
    "resolution": "客服代表指导顾客重新校准恒温器以匹配室温。",
    "followUpRequired": true,
    "followUpDetails": "顾客需在一小时后检查，看重新校准是否解决了温度问题。"
  },
  "status": "COMPLETE",
  "ambiguities": []
}
</json>
```

In [ ]:
summarize_call_with_improved_prompt(call3)

```txt
<thinking>
主要问题：顾客的Acme SecureHome系统警报在半夜随机响起，即使门窗都已正确关闭。

解决方案：客服代表建议对系统运行诊断程序，以查明潜在的传感器故障。顾客被转接到技术团队，以执行诊断并解决问题。

需要后续跟进：是的，技术团队需要跟进顾客，诊断并修复警报系统问题。

模糊之处：对话中未发现。
</thinking>

<json>
{
  "summary": {
    "customerIssue": "顾客的家庭安全警报系统在夜间无故随机响起。",
    "resolution": "客服代表建议运行诊断程序检查传感器故障，并将顾客转接给技术团队。",
    "followUpRequired": true,
    "followUpDetails": "技术团队将诊断并解决顾客警报系统的问题。"
  },
  "status": "COMPLETE",
  "ambiguities": []
}
</json>
```

这些回复看起来都很棒！让我们再尝试一个带有一些模糊性的通话记录，看看JSON结果是否会包含这些模糊之处：

In [ ]:
ambiguous_call = """
客服代表：感谢您致电Acme智能家居支持。我是亚历克斯。今天有什么可以帮您的吗？
顾客：嗨，亚历克斯，我的智能门锁（SmartLock）出了点问题。它工作不正常。
客服代表：很抱歉听到这个问题。您能告诉我您的智能门锁具体发生了什么吗？
顾客：嗯，有时候我出门的时候它不会自动上锁。我觉得可能和我的手机有关，但我不确定。
客服代表：我明白了。您说它不锁，是指它对自动上锁功能没有反应，还是您尝试通过应用程序手动上锁时没有反应？
顾客：呃，我想两者都有。有时候一个能用，有时候另一个能用。它很不稳定。
客服代表：好的。您提到可能和您的手机有关。您有没有注意到什么规律，比如离门近一点的时候它工作得更好？
顾客：也许吧？我没怎么注意过那个。
客服代表：好的。我们来尝试排查一下故障。首先，您能告诉我您智能门锁的型号吗？
顾客：我不确定。我大概六个月前买的，如果这有帮助的话。
客服代表：没关系。您能在锁本身上看到型号吗？
顾客：我得去看看。我们能直接假设它是最新型号吗？
客服代表：嗯，知道确切型号会帮助我们更有效地排查故障。但我们还是继续用我们已知的信息。您最近尝试重置过锁吗？
顾客：我想是的。或者那可能是我的智能恒温器。我那个也一直有问题。
客服代表：我明白了。听起来我们可能需要对您的智能门锁进行一次全面的诊断。您现在方便我引导您完成这个过程吗？
顾客：实际上，我得去赴个约。我能稍后回电吗？
客服代表：当然。在您离开之前，有没有一个好的联系电话，我们的技术团队可以联系您进行更深入的故障排除会话？
顾客：当然，您可以打555……哦等等，那是我的旧号码。我查一下我的新号码……算了，我还是等有空的时候再打回来吧。
客服代表：我明白了。我们24/7全天候为您服务，随时准备为您排查故障。在您离开之前，还有其他我可以帮助您的吗？
顾客：没有了，就这些。谢谢。
客服代表：不客气。感谢您选择Acme智能家居。祝您有愉快的一天！
"""

summarize_call_with_improved_prompt(ambiguous_call)

```txt
<thinking>
主要问题：顾客的Acme智能门锁（SmartLock）无法通过自动上锁功能或应用程序进行手动上锁，表现出不一致性。

解决方案：客服代表尝试通过询问具体的智能门锁型号并建议重置来排查故障，但顾客在完成故障排除过程之前不得不离开。

需要后续跟进：是的，顾客需要回电，以便与技术团队完成全面的诊断和故障排除会话。

模糊之处：
* 顾客不确定问题是否与他们的手机有关，这暗示可能存在连接问题。
* 顾客提到其他Acme产品（智能恒温器）也有问题，但尚不清楚这些问题是否与智能门锁问题相关。
* 顾客的联系电话没有明确提供，这可能会使后续跟进变得更加困难。
</thinking>

<json>
{
  "summary": {
    "customerIssue": "智能门锁无法通过自动或手动方式稳定上锁。",
    "resolution": "尝试排查故障，但顾客在完成过程前离开。",
    "followUpRequired": true,
    "followUpDetails": "顾客需回电与技术团队进行全面诊断和故障排除会话。"
  },
  "status": "COMPLETE",
  "ambiguities": [
    "可能与顾客手机存在连接问题",
    "不清楚是否与其他Acme产品问题相关",
    "顾客联系电话未明确提供"
  ]
}
</json>
```

太棒了！一切似乎都按预期工作。

---

## 边缘情况

到目前为止，我们尝试过的所有通话记录都是相对直接的客户服务电话。在现实世界中，我们预计还会遇到一些我们可能不希望进行总结的通话记录，包括：

*   有连接问题的通话
*   有语言障碍的通话
*   有乱码文本记录的通话
*   有不理智或情绪激动顾客的通话

请记住，我们的目标是总结这些通话，以帮助衡量我们提供的客户服务的有效性。如果我们将这些边缘情况的通话也包含在摘要中，我们很可能会得到偏差的结果。

让我们看看当前提示在处理这些边缘情况时会发生什么。下面我们定义了一些新的通话记录：

In [ ]:
wrong_number_call = """
客服代表：Acme智能家居支持，莉萨为您服务。有什么可以帮您的吗？
顾客：这是技术支持吗？
客服代表：是的，这里是Acme智能家居设备的技术支持。有什么可以帮您的吗？
顾客：抱歉，打错了。
客服代表：没问题。祝您有愉快的一天。
"""

incomplete_call = """
客服代表：Acme智能家居支持，我是莎拉。今天有什么可以帮您的吗？
顾客：那个东西不工作了。
客服代表：很抱歉听到这个问题。您能具体说明是哪个设备出了问题吗？
顾客：你知道的，就是那个常用的。我得走了，再见。
客服代表：等等，我需要更多信息……[通话已断开]
"""

garbled_call = """
客服代表：感谢您致电Acme智能家居支持。我是亚历克斯。有什么可以帮您的吗？
顾客：[声音模糊不清]
客服代表：喂？您还在吗？
"""

language_barrier_call = """
客服代表：Acme智能家居支持，莎拉为您服务。有什么可以帮您的吗？
顾客：[说西班牙语]
客服代表：很抱歉，我不会说西班牙语。您会说英语吗？
顾客：[继续说西班牙语]
客服代表：请稍等，我尝试找一位翻译员接听电话……
"""

让我们将这些边缘情况的通话记录通过我们的提示运行，看看会得到什么样的结果：

In [ ]:
summarize_call_with_improved_prompt(wrong_number_call)

```txt
<thinking>
问题：顾客似乎拨错了技术支持电话。
解决方案：由于是打错了电话，没有需要解决的问题。客服代表礼貌地结束了通话。
后续跟进：由于是拨错电话，不需要后续跟进。
模糊之处：这段简短的对话中没有明显的模糊之处。
</thinking>

<json>
{
  "summary": {
    "customerIssue": "顾客拨错了技术支持电话",
    "resolution": "客服代表礼貌地结束了通话，因为是打错了号码",
    "followUpRequired": false,
    "followUpDetails": null
  },
  "status": "COMPLETE",
  "ambiguities": []
}
</json>
```

In [ ]:
summarize_call_with_improved_prompt(incomplete_call)

```txt
<thinking>
在这段通话记录中，主要问题不明确，因为顾客没有提供他们遇到问题的设备的足够细节。客服代表试图澄清问题，但顾客在提供更多信息之前就挂断了电话。

由于问题没有明确识别，因此没有解决方案。然而，需要后续跟进以从顾客那里收集关于具体设备和他们遇到的问题的更多细节。

关键的模糊之处在于设备和问题性质的不明确。顾客模糊的陈述（“那个东西不工作了”和“就是那个常用的”）没有提供足够的信息让客服代表诊断或解决问题。
</thinking>

<json>
{
  "summary": {
    "customerIssue": "顾客报告某个未指明的设备不工作，但未提供进一步细节。",
    "resolution": "由于顾客缺乏信息，无法解决问题。",
    "followUpRequired": true,
    "followUpDetails": "客服代表需要再次联系顾客，以收集关于具体设备和问题的细节。"
  },
  "status": "COMPLETE",
  "ambiguities": ["顾客所指的设备", "设备问题的性质"]
}
</json>
```

In [ ]:
summarize_call_with_improved_prompt(garbled_call)

```txt
<thinking>
从通话记录来看，顾客似乎致电了Acme智能家居支持，但他们的声音模糊不清，客服代表无法理解。由于信息有限，不清楚主要问题或致电原因是什么。没有提供解决方案，并且是否需要后续跟进也存在模糊之处，因为问题本身是未知的。关键的模糊之处在于顾客缺乏清晰的沟通，导致客服代表无法理解问题。
</thinking>

<json>
{
  "summary": {
    "customerIssue": "因顾客声音模糊不清，问题不明确",
    "resolution": "由于无法理解问题，未提供解决方案",
    "followUpRequired": true,
    "followUpDetails": "客服代表应尝试重新联系顾客以进行更清晰的沟通"
  },
  "status": "COMPLETE",
  "ambiguities": ["顾客声音模糊不清，导致无法理解问题"]
}
</json>
```

In [ ]:
summarize_call_with_improved_prompt(language_barrier_call)

```txt
<thinking>
主要问题：顾客致电并说西班牙语，但客服代表不懂西班牙语。
解决方案：客服代表尝试找一位翻译员接听电话，以解决语言障碍。
需要后续跟进：是的，客服代表需要联系一位西班牙语翻译员来协助顾客。
模糊之处：不清楚顾客致电的原因，因为通话记录中没有说明。
</thinking>

<json>
{
  "summary": {
    "customerIssue": "顾客使用客服代表不理解的语言（西班牙语）进行交流。",
    "resolution": "客服代表尝试找翻译员解决语言障碍。",
    "followUpRequired": true,
    "followUpDetails": "客服代表需要为顾客联系一位西班牙语翻译员。"
  },
  "status": "COMPLETE",
  "ambiguities": ["通话记录中未说明顾客致电的原因。"]
}
</json>
```

不幸的是，我们正在为这些边缘情况的通话记录获得完整的摘要。以下是响应中一些有问题的地方：

> "customerIssue": "顾客使用客服代表不理解的语言（西班牙语）进行交流。"

> "customerIssue": "因顾客声音模糊不清，问题不明确"

> "customerIssue": "顾客拨错了技术支持电话"

请记住，我们的目标是总结这些客户服务电话，以深入了解我们的客户服务团队的效率。这些边缘情况的通话记录导致了完整的摘要，这在分析所有摘要时会引起问题。我们需要决定一个处理这些通话的策略。

---

## 进一步的提示改进

正如我们之前所见，我们的提示目前正在为边缘情况的通话记录生成完整的摘要。我们希望改变这种行为。对于如何处理这些边缘情况，我们有几种选择：

*   以某种方式标记它们，表明它们不可摘要，以便后续进行人工审查。
*   将它们单独分类（例如，“技术困难”、“语言障碍”等）。

为了简化起见，我们将选择通过要求模型输出如下所示的JSON来标记这些边缘情况的通话：

```json
{
  "status": "INSUFFICIENT_DATA"
}
```

为了实现这一点，我们需要通过以下方式更新我们的提示：
*   添加说明，解释所需的“INSUFFICIENT_DATA”输出。
*   添加示例，展示可摘要和不可摘要的通话记录及其相应的JSON输出。

### 更新我们的指令

让我们编写提示指令部分的新内容，以解释模型何时应该输出我们的“INSUFFICIENT_DATA”JSON。

In [ ]:
# 仅为新增内容。我们稍后会查看整个提示
new_instructions_addition = """
数据不足标准：
   如果满足以下任一条件：
   a) 通话记录的总交流次数少于5次，或者
   b) 顾客的问题不明确，或者
   c) 通话内容模糊不清、不完整或受到语言障碍的阻碍
   则只返回以下JSON：
   {
     "status": "INSUFFICIENT_DATA"
   }
"""

### 添加示例

正如我们之前在本课程中讨论过的，在提示中添加示例几乎总是一个好主意。在这个特定的用例中，示例将帮助Claude大致理解我们希望为可摘要和不可摘要的通话记录生成的摘要类型。

以下是我们可以在提示中包含的一组示例：

In [ ]:
examples_for_prompt = """
<examples>
1. 完整交互：
<transcript>
客服代表：感谢您致电Acme智能家居支持。我是亚历克斯。有什么可以帮您的吗？
顾客：嗨，亚历克斯，我的Acme智能恒温器（SmartTherm）无法保持我设定的温度。它设定的是72度，但房子里要热得多。
客服代表：很抱歉听到这个问题。我们来排查一下故障。您的智能恒温器连接到Wi-Fi了吗？
顾客：是的，显示屏上显示着Wi-Fi符号。
客服代表：好的。我们来重新校准您的智能恒温器。请按住菜单按钮5秒钟。
顾客：好的，按了。出现了一个新菜单。
客服代表：太好了。导航到“校准”（Calibration）并按下选择键。调整温度以匹配您房间温度计的读数。
顾客：好的，我已经调到79度来匹配了。
客服代表：很好。按下选择键确认。它将进行重新校准，这可能需要几分钟。一小时后回来检查一下是否已修复。
顾客：好的，我会照做的。谢谢你的帮助，亚历克斯。
客服代表：不客气！今天还有其他我可以帮助您的吗？
顾客：没有了，就这些。再次感谢。
客服代表：感谢您选择Acme智能家居。祝您有愉快的一天！
</transcript>

<thinking>
主要问题：智能恒温器无法保持设定温度
解决方案：指导顾客完成重新校准过程
后续跟进：不需要，但顾客应在一小时后检查效果
模糊之处：未发现
</thinking>

<json>
{
  "summary": {
    "customerIssue": "智能恒温器无法保持设定温度，显示高于设定的72度",
    "resolution": "指导顾客完成智能恒温器重新校准过程",
    "followUpRequired": false,
    "followUpDetails": null
  },
  "status": "COMPLETE",
  "ambiguities": []
}
</json>

2. 需要后续跟进的交互：
<transcript>
客服代表：Acme智能家居支持，我是杰米。有什么可以帮您的吗？
顾客：嗨，我刚安装了新的Acme智能摄像头（SmartCam），但无法连接到Wi-Fi。
客服代表：我很乐意帮助您。您正在使用Acme智能家居应用程序吗？
顾客：是的，我的手机上有这个应用程序。
客服代表：好的。请确保您的手机连接的是2.4GHz的Wi-Fi网络，而不是5GHz的。
顾客：哦，我连接的是5GHz网络。我应该切换吗？
客服代表：是的，请切换到2.4GHz网络。智能摄像头只支持2.4GHz。
顾客：好的，完成了。现在怎么办？
客服代表：打开应用程序，选择“添加设备”，选择“智能摄像头”，然后按照屏幕上的指示操作。
顾客：它现在要求输入密码。
客服代表：输入您的Wi-Fi密码，它应该就能连接了。
顾客：还是不行。我一直收到错误信息。
客服代表：我明白了。在这种情况下，我想将此问题升级给我们的技术团队。他们会在24小时内联系您。
顾客：好的，听起来不错。谢谢您尝试帮助我。
客服代表：不客气。还有其他需要帮助的吗？
顾客：没有了，目前就这些。再次感谢。
客服代表：感谢您选择Acme智能家居。祝您有愉快的一天！
</transcript>

<thinking>
主要问题：顾客无法将新的智能摄像头连接到Wi-Fi
解决方案：初步故障排除不成功，问题已升级给技术团队
后续跟进：需要，技术团队将在24小时内联系顾客
模糊之处：未提及顾客收到的具体错误信息
</thinking>

<json>
{
  "summary": {
    "customerIssue": "无法将新的智能摄像头连接到Wi-Fi",
    "resolution": "初步故障排除不成功，问题已升级给技术团队",
    "followUpRequired": true,
    "followUpDetails": "技术团队将在24小时内联系顾客提供进一步帮助"
  },
  "status": "COMPLETE",
  "ambiguities": ["未提及顾客收到的具体错误信息"]
}
</json>

3. 数据不足：
<transcript>
客服代表：Acme智能家居支持，我是萨姆。有什么可以帮您的吗？
顾客：嗨，我的智能门锁不工作了。
客服代表：很抱歉听到这个问题。您能告诉我更多关于这个问题的信息吗？
顾客：它就是不工作。我不知道还能说什么。
客服代表：好的，您是什么时候第一次注意到这个问题的？您用的是什么型号的Acme智能门锁？
顾客：我不记得了。听着，我得走了。我稍后回电。
客服代表：好的，如果您需要进一步帮助，我们24/7全天候为您服务。祝您有愉快的一天。
</transcript>

<thinking>
这段通话记录的交流次数少于5次，且顾客的问题不明确。顾客没有提供关于智能门锁问题的具体细节，也没有回答客服代表的问题。这次交互没有提供足够的信息来生成完整的摘要。
</thinking>

<json>
{
  "status": "INSUFFICIENT_DATA"
}
</json>
</examples>
"""

请注意，这些示例涵盖了三种不同的情况：
*   一个不需要后续跟进的完整交互
*   一个需要后续跟进且包含模糊之处的完整交互
*   一个包含数据不足的不可摘要交互

在向Claude提供示例时，涵盖各种输入/输出对是很重要的。

---

## 我们的最终提示

让我们将初始提示与上一节中添加的内容结合起来：
*   关于处理数据不足通话的指令
*   一组示例输入和输出

这是新的完整提示：

In [ ]:
system = """
您是一位专业的客户服务分析师，擅长从通话记录中提取关键信息并以结构化格式进行总结。
您的任务是分析客户服务通话记录，并生成简洁、准确的摘要，同时保持专业的语气。
"""

prompt = """
分析以下客户服务通话记录，并生成一份交互的JSON格式摘要：

<transcript>
[在此处插入通话记录]
</transcript>

指令：
<instructions>
1. 仔细阅读通话记录。
2. 分析通话记录，重点关注主要问题、解决方案以及任何需要后续跟进的事项。
3. 根据指定结构生成一个JSON对象，总结交互的关键方面。

重要指南：
- 保密性：省略所有具体的客户数据，如姓名、电话号码和电子邮件地址。
- 字符限制：将每个文本字段限制在最多100个字符。
- 在您的摘要中保持专业的语气。

输出格式：
生成一个具有以下结构的JSON对象：
<json>
{
  "summary": {
    "customerIssue": "主要问题或致电原因的简要描述",
    "resolution": "问题如何处理或解决（如果适用）",
    "followUpRequired": true/false,
    "followUpDetails": "任何必要的后续行动的描述，如果不需要则为null"
  },
  "status": "COMPLETE",
  "ambiguities": ["对话中任何不清楚或模糊点的列表，如果没有则为空数组"]
}
</json>

数据不足标准：
   如果满足以下任一条件：
   a) 通话记录的总交流次数少于5次
   b) 顾客的问题不明确
   c) 通话内容模糊不清、不完整或受到语言障碍的阻碍
   则只返回以下JSON：
   {
     "status": "INSUFFICIENT_DATA"
   }

示例：
<examples>
1. 完整交互：
<transcript>
客服代表：感谢您致电Acme智能家居支持。我是亚历克斯。有什么可以帮您的吗？
顾客：嗨，亚历克斯，我的Acme智能恒温器（SmartTherm）无法保持我设定的温度。它设定的是72度，但房子里要热得多。
客服代表：很抱歉听到这个问题。我们来排查一下故障。您的智能恒温器连接到Wi-Fi了吗？
顾客：是的，显示屏上显示着Wi-Fi符号。
客服代表：好的。我们来重新校准您的智能恒温器。请按住菜单按钮5秒钟。
顾客：好的，按了。出现了一个新菜单。
客服代表：太好了。导航到“校准”（Calibration）并按下选择键。调整温度以匹配您房间温度计的读数。
顾客：好的，我已经调到79度来匹配了。
客服代表：很好。按下选择键确认。它将进行重新校准，这可能需要几分钟。一小时后回来检查一下是否已修复。
顾客：好的，我会照做的。谢谢你的帮助，亚历克斯。
客服代表：不客气！今天还有其他我可以帮助您的吗？
顾客：没有了，就这些。再次感谢。
客服代表：感谢您选择Acme智能家居。祝您有愉快的一天！
</transcript>

<thinking>
主要问题：智能恒温器无法保持设定温度
解决方案：指导顾客完成重新校准过程
后续跟进：不需要，但顾客应在一小时后检查效果
模糊之处：未发现
</thinking>

<json>
{
  "summary": {
    "customerIssue": "智能恒温器无法保持设定温度，显示高于设定的72度",
    "resolution": "指导顾客完成智能恒温器重新校准过程",
    "followUpRequired": false,
    "followUpDetails": null
  },
  "status": "COMPLETE",
  "ambiguities": []
}
</json>

2. 需要后续跟进的交互：
<transcript>
客服代表：Acme智能家居支持，我是杰米。有什么可以帮您的吗？
顾客：嗨，我刚安装了新的Acme智能摄像头（SmartCam），但无法连接到Wi-Fi。
客服代表：我很乐意帮助您。您正在使用Acme智能家居应用程序吗？
顾客：是的，我的手机上有这个应用程序。
客服代表：好的。请确保您的手机连接的是2.4GHz的Wi-Fi网络，而不是5GHz的。
顾客：哦，我连接的是5GHz网络。我应该切换吗？
客服代表：是的，请切换到2.4GHz网络。智能摄像头只支持2.4GHz。
顾客：好的，完成了。现在怎么办？
客服代表：打开应用程序，选择“添加设备”，选择“智能摄像头”，然后按照屏幕上的指示操作。
顾客：它现在要求输入密码。
客服代表：输入您的Wi-Fi密码，它应该就能连接了。
顾客：还是不行。我一直收到错误信息。
客服代表：我明白了。在这种情况下，我想将此问题升级给我们的技术团队。他们会在24小时内联系您。
顾客：好的，听起来不错。谢谢您尝试帮助我。
客服代表：不客气。还有其他需要帮助的吗？
顾客：没有了，目前就这些。再次感谢。
客服代表：感谢您选择Acme智能家居。祝您有愉快的一天！
</transcript>

<thinking>
主要问题：顾客无法将新的智能摄像头连接到Wi-Fi
解决方案：初步故障排除不成功，问题已升级给技术团队
后续跟进：需要，技术团队将在24小时内联系顾客
模糊之处：未提及顾客收到的具体错误信息
</thinking>

<json>
{
  "summary": {
    "customerIssue": "无法将新的智能摄像头连接到Wi-Fi",
    "resolution": "初步故障排除不成功，问题已升级给技术团队",
    "followUpRequired": true,
    "followUpDetails": "技术团队将在24小时内联系顾客提供进一步帮助"
  },
  "status": "COMPLETE",
  "ambiguities": ["未提及顾客收到的具体错误信息"]
}
</json>

3. 数据不足：
<transcript>
客服代表：Acme智能家居支持，我是萨姆。有什么可以帮您的吗？
顾客：嗨，我的智能门锁不工作了。
客服代表：很抱歉听到这个问题。您能告诉我更多关于这个问题的信息吗？
顾客：它就是不工作。我不知道还能说什么。
客服代表：好的，您是什么时候第一次注意到这个问题的？您用的是什么型号的Acme智能门锁？
顾客：我不记得了。听着，我得走了。我稍后回电。
客服代表：好的，如果您需要进一步帮助，我们24/7全天候为您服务。祝您有愉快的一天。
</transcript>

<thinking>
这段通话记录的交流次数少于5次，且顾客的问题不明确。顾客没有提供关于智能门锁问题的具体细节，也没有回答客服代表的问题。这次交互没有提供足够的信息来生成完整的摘要。
</thinking>

<json>
{
  "status": "INSUFFICIENT_DATA"
}
</json>
</examples>
</instructions>

在生成JSON之前，请在<thinking>标签中分析通话记录。
请包含您对主要问题、解决方案、后续跟进要求以及任何模糊之处的识别。
然后，在<json>标签中提供您的JSON输出。
"""

上面的提示相当长，但其总体结构如下：
*   系统提示设定了模型的上下文、角色和语气。
*   主提示包含以下内容：
    *   通话记录
    *   一组指令，其中包含：
        *   通用指令
        *   指导方针
        *   输出格式要求
        *   处理边缘情况通话的详细信息
        *   示例
    *   关于在输出中使用XML标签的详细信息

以下是一个摘要，以帮助可视化提示的流程：

```txt
分析以下客户服务通话记录，并生成一份交互的JSON格式摘要：

<transcript>
[在此处插入通话记录]
</transcript>

<instructions>
- 通用指令和指导方针
- 输出JSON格式描述
- 数据不足（边缘情况）标准
<examples>
各种示例输入和输出
</examples>
</instructions>

在生成JSON之前，请在<thinking>标签中分析通话记录。
请包含您对主要问题、解决方案、后续跟进要求以及任何模糊之处的识别。
然后，在<json>标签中提供您的JSON输出。
```

让我们用一个新函数来测试最终的提示。请注意，这个函数会提取 `<json>` 标签内的JSON摘要内容：

In [ ]:
import re

def summarize_call_with_final_prompt(transcript):
    final_prompt = prompt.replace("[INSERT CALL TRANSCRIPT HERE]", transcript)
    # Make the API call
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        system=system,
        max_tokens=4096,
        messages=[
            {"role": "user", "content": final_prompt}
        ]
    )
    
    # Extract content between <json> tags
    json_content = re.search(r'<json>(.*?)</json>', response.content[0].text, re.DOTALL)
    
    if json_content:
        print(json_content.group(1).strip())
    else:
        print("No JSON content found in the response.")

让我们用我们现有的一些通话变量来测试一下：

In [ ]:
summarize_call_with_final_prompt(call1)

```txt
{
  "summary": {
    "customerIssue": "无法打开智能灯泡",
    "resolution": "客服代表指导顾客通过断电再通电的方式重置灯泡",
    "followUpRequired": false,
    "followUpDetails": null
  },
  "status": "COMPLETE",
  "ambiguities": []
}
```

In [ ]:
summarize_call_with_final_prompt(call3)

```txt
{
  "summary": {
    "customerIssue": "Acme SecureHome警报系统在夜间无故随机多次响起",
    "resolution": "已采取初步故障排除步骤，但问题未解决。顾客已转接给技术团队进行诊断",
    "followUpRequired": true,
    "followUpDetails": "技术团队将诊断并解决警报系统问题"
  },
  "status": "COMPLETE",
  "ambiguities": []
}
```

In [ ]:
summarize_call_with_final_prompt(call3)

让我们尝试一个通话记录，它应该会生成一个 `ambiguities` 数组非空的摘要：

In [ ]:
summarize_call_with_final_prompt(ambiguous_call)

```txt
{
  "summary": {
    "customerIssue": "智能门锁无法通过自动或应用程序稳定上锁，行为不一致",
    "resolution": "尝试排查故障，但因缺乏型号细节且顾客需离开而未完成",
    "followUpRequired": true,
    "followUpDetails": "顾客需在有空时回电，以便进一步排查智能门锁问题"
  },
  "status": "COMPLETE",
  "ambiguities": [
    "不清楚是否提及了相关的智能恒温器问题",
    "智能门锁型号未识别",
    "顾客联系电话未确认"
  ]
}
```

现在，让我们尝试一些我们不希望生成摘要的边缘情况提示：

In [ ]:
summarize_call_with_final_prompt(garbled_call)

{
  "status": "INSUFFICIENT_DATA"
}


In [ ]:
summarize_call_with_final_prompt(language_barrier_call)

{
  "status": "INSUFFICIENT_DATA"
}


In [ ]:
summarize_call_with_final_prompt(incomplete_call)

{
  "status": "INSUFFICIENT_DATA"
}


太棒了！我们得到了我们想要的精确输出！让我们尝试更进一步：

In [ ]:
summarize_call_with_final_prompt("blah blah blah")

{
  "status": "INSUFFICIENT_DATA"
}


In [ ]:
summarize_call_with_final_prompt("")

{
  "status": "INSUFFICIENT_DATA"
}


太棒了，这个提示正在处理我们所有的边缘情况！

---

## 总结

在本课程中，我们详细讲解了开发一个用于总结客户服务通话记录的复杂提示的过程。让我们回顾一下我们所采用的提示技巧：

* 系统提示：我们使用系统提示为Claude设定了整体上下文和角色。
* 结构化输入：我们使用XML标签将通话记录放在提示的开头。
* 清晰指令：我们提供了详细的指导，说明要关注什么以及如何构建输出。
* 输出格式化：我们为摘要指定了JSON结构，确保结果一致且易于解析。
* 处理边缘情况：我们添加了识别数据不足通话的标准。
* 示例：我们包含了多样化的示例，以说明不同场景下的期望输出。
* 大声思考：我们要求Claude在提供最终JSON输出之前，在`<thinking>`标签中展示其分析过程。

通过采用这些技巧，我们创建了一个强大的提示，它能够为各种客户服务通话记录生成结构化摘要，同时恰当地处理边缘情况。这种方法可以适用于许多其他复杂的提示场景，而不仅仅是通话摘要。

**重要提示**：尽管我们开发了一个看似能够很好地处理我们测试用例的复杂提示，但至关重要的是要理解，这个提示尚未达到生产就绪状态。我们所创建的只是一个有希望的起点，但在其能够可靠地用于实际环境之前，还需要进行大量的测试和评估。我们目前的“肉眼”测试评估是基于一小组示例进行的。这并不能代表真实客户服务电话的多样性和往往不可预测的性质。为了确保提示的有效性和可靠性，我们需要实施一个包含定量指标的全面评估过程。稳健的、数据驱动的评估是弥合有前景的原型与可靠的、生产级解决方案之间差距的关键。

# Lesson 4: Call transcript summarizer

In this lesson, we're going to write a complex prompt for a common customer use-case: summarizing.  Specifically, we'll summarize long customer service call transcripts.  Our goal is to summarize customer service calls for customer support metrics.  We want summaries of complete customer service calls to evaluate the efficacy of our customer support team.  This means we'll exclude calls that have connection issues, language barriers, and other issues that hinder effective summarization.

Let's imagine we work for Acme Corporation, a company that sells smart home devices. The company handles hundreds of customer service calls daily and needs a way to quickly turn these conversations into **useful, structured data**. 

Some important considerations include:
* Calls can be short and sweet or long and complicated.
* Customers might be calling about anything from a simple Wi-Fi connection issue to a complex system malfunction.
* We need our summaries in a specific format so they're easy to analyze later.
* We have to be careful not to include any personal customer information in our summaries.

To help us out, we'll follow the best practices we described previously:
* Use a system prompt to set the stage.
* Structure the prompt for optimal performance.
* Give clear instructions and define your desired output.
* Use XML tags to organize information.
* Handle special cases and edge scenarios.
* Provide examples to guide the model.

---

## Understanding the data

Now that we understand our task, let's take a look at the data we'll be working with. In this lesson, we'll use a variety of simulated customer service call transcripts from Acme Corporation's smart home device support team. These transcripts will help us create a robust prompt that can handle different scenarios.

Let's examine some of the types of call transcripts we might encounter:

A short and simple transcript:

In [42]:
call1 = """
Agent: Thank you for calling Acme Smart Home Support. This is Alex. How can I help you?
Customer: Hi, I can't turn on my smart light bulb.
Agent: I see. Have you tried resetting the bulb?
Customer: Oh, no. How do I do that?
Agent: Just turn the power off for 5 seconds, then back on. It should reset.
Customer: Ok, I'll try that. Thanks!
Agent: You're welcome. Call us back if you need further assistance.
"""

A medium-length transcript with an eventual resolution:

In [43]:
call2 = """
Agent: Acme Smart Home Support, this is Jamie. How may I assist you today?
Customer: Hi Jamie, my Acme SmartTherm isn't maintaining the temperature I set. It's set to 72 but the house is much warmer.
Agent: I'm sorry to hear that. Let's troubleshoot. Is your SmartTherm connected to Wi-Fi?
Customer: Yes, the Wi-Fi symbol is showing on the display.
Agent: Great. Let's recalibrate your SmartTherm. Press and hold the menu button for 5 seconds.
Customer: Okay, done. A new menu came up.
Agent: Perfect. Navigate to "Calibration" and press select. Adjust the temperature to match your room thermometer.
Customer: Alright, I've set it to 79 degrees to match.
Agent: Great. Press select to confirm. It will recalibrate, which may take a few minutes. Check back in an hour to see if it's fixed.
Customer: Okay, I'll do that. Thank you for your help, Jamie.
Agent: You're welcome! Is there anything else I can assist you with today?
Customer: No, that's all. Thanks again.
Agent: Thank you for choosing Acme Smart Home. Have a great day!
"""

A longer call with no resolution:

In [44]:
call3 = """
Agent: Thank you for contacting Acme Smart Home Support. This is Sarah. How can I help you today?
Customer: Hi Sarah, I'm having trouble with my Acme SecureHome system. The alarm keeps going off randomly.
Agent: I'm sorry to hear that. Can you tell me when this started happening?
Customer: It started about two days ago. It's gone off three times now, always in the middle of the night.
Agent: I see. Are there any error messages on the control panel when this happens?
Customer: No, I didn't notice any. But I was pretty groggy each time.
Agent: Understood. Let's check a few things. First, can you confirm that all your doors and windows are closing properly?
Customer: Yes, I've checked all of them. They're fine.
Agent: Okay. Next, let's check the battery in your control panel. Can you tell me if the low battery indicator is on?
Customer: Give me a moment... No, the battery indicator looks normal.
Agent: Alright. It's possible that one of your sensors is malfunctioning. I'd like to run a diagnostic, but I'll need to transfer you to our technical team for that. Is that okay?
Customer: Yes, that's fine. I just want this fixed. It's really disruptive.
Agent: I completely understand. I'm going to transfer you now. They'll be able to run a full system diagnostic and hopefully resolve the issue for you.
Customer: Okay, thank you.
Agent: You're welcome. Thank you for your patience, and I hope you have a great rest of your day.
"""

These examples showcase the variety of calls and considerations we need to handle:
* Calls have wildly different lengths.
* Calls feature various support issues (simple fixes, device malfunctions, complex problems).
* Some calls end with a resolution and others remain unresolved cases.
* Some calls require follow-up.

As we build our prompt, we'll need to ensure it can effectively summarize all these types of calls, extracting the key information and presenting it in a consistent, structured format.
In the next section, we'll start building our prompt, step by step, to handle this diverse range of call transcripts. 

---

## A simple version of the prompt
Now that we understand our task and the kind of data we're working with, let's start building our prompt. We'll begin with a basic version and gradually refine it to handle the complexities of our call summarization task.

Let's begin with this very simple prompt that outlines the basic task:

In [45]:
prompt = """
Summarize the following customer service call transcript. Focus on the main issue, how it was resolved, and any required follow-up.

{transcript}
"""

This basic prompt gives Claude a general idea of what we want, but it has several limitations:

* It doesn't specify the desired output format, which could lead to inconsistent summaries.
* It doesn't provide guidance on how to handle different scenarios (like unresolved issues or insufficient information).
* It doesn't set any constraints on length or content, potentially resulting in overly long or detailed summaries.
* It doesn't instruct Claude to omit personal information, which could lead to privacy issues.

With that said, let's test it out to get a sense of how it performs:

In [46]:
from anthropic import Anthropic
from dotenv import load_dotenv

load_dotenv()
client = Anthropic()

def summarize_call(transcript):
    final_prompt = prompt.format(transcript=transcript)
    # Make the API call
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        messages=[
            {"role": "user", "content": final_prompt}
        ]
    )
    print(response.content[0].text)

In [47]:
summarize_call(call1)

Here is a summary of the customer service call transcript:

Main Issue:
The customer was unable to turn on their Acme smart light bulb.

Resolution:
The service agent instructed the customer to reset the bulb by turning the power off for 5 seconds and then back on. This should reset the bulb and allow it to turn on.

Follow-Up:
The agent told the customer to call back if they continued to have issues after trying the reset procedure. No other follow-up was mentioned.


In [48]:
summarize_call(call2)

Summary:

Main Issue: The customer's Acme SmartTherm thermostat was not maintaining the set temperature of 72°F, and the house was much warmer.

Resolution: The agent guided the customer through the process of recalibrating the SmartTherm thermostat. This involved accessing the "Calibration" menu, adjusting the temperature to match the customer's room thermometer (79°F in this case), and confirming the new setting. The recalibration process may take a few minutes to complete.

Follow-up Required: The customer was advised to check the thermostat in an hour to see if the issue was resolved after the recalibration process completed.


In [49]:
summarize_call(call3)

Here is a summary of the customer service call transcript:

Main Issue:
The customer was having an issue with their Acme SecureHome alarm system going off randomly in the middle of the night, even though all doors and windows were closed properly.

How It Was Resolved:
The customer service agent first had the customer check for any error messages on the control panel and confirm that the battery was not low. When those basic troubleshooting steps did not reveal the issue, the agent determined that one of the sensors may be malfunctioning and needed to transfer the customer to the technical support team for a full system diagnostic.

Required Follow-Up:
The technical support team needs to run a diagnostic on the customer's SecureHome system to identify which sensor(s) may be causing the false alarms and then repair or replace those components. The customer should be contacted again once the diagnostic is complete and the repair/replacement has been performed to ensure the random alarms 

As you can see, while Claude does provide a summary, it's not in a format that would be easy to analyze systematically. The summary might be too long or too short, and it might not consistently cover all the points we're interested in.


In the next steps, we'll start adding more structure and guidance to our prompt to address these limitations. We'll see how each addition improves the quality and consistency of Claude's summaries.

Remember, prompt engineering is an iterative process. We start simple and gradually refine our prompt.

---

## Adding a system prompt

The easiest place to start is with a system prompt that sets the overall context and role for Claude, helping to guide its behavior throughout the interaction.

Let's start with this system prompt:

In [50]:
system = """
You are an expert customer service analyst, skilled at extracting key information from call transcripts and summarizing them in a structured format.
Your task is to analyze customer service call transcripts and generate concise, accurate summaries while maintaining a professional tone.
"""

--- 

## Structuring our main prompt

Next, we're going to start writing the main prompt.  We'll rely on some of these prompting tips:

- Put long documents (our transcripts) at the top.
- Add detailed instructions and output format requirements.
- Introduce XML tags for structuring the prompt and output.
- Give Claude space "to think out loud". 

Because this prompt may get quite long, we'll write individual pieces in isolation and then combine them together.


### The input data
When working with large language models like Claude, it's crucial to put long documents, like our call transcripts, at the beginning of the prompt. This ensures that Claude has all the necessary context before receiving specific instructions. We should also use XML tags to identify the transcript in the prompt:

In [51]:
prompt_pt1 = """
Analyze the following customer service call transcript and generate a JSON summary of the interaction:

<transcript>
[INSERT CALL TRANSCRIPT HERE]
</transcript>
"""

### Instructions and output format

Before we go any further, let's think clearly about what a good structured output format might look like.  To make our life easier when parsing the results, it's often easiest to ask Claude for a JSON response.  What should a good JSON look like in this case?

At a minimum, our JSON output should include the following:
- A status as to whether Claude had enough information to generate a summary.  We'll come back to this.  For now, we'll assume that all summaries have a status of "COMPLETE" meaning that Claude could generate a summary.
- A summary of the customer issue
- If the call requires additional follow up
- Details on any follow up actions, if required (call the customer back, etc.)
- How the issue was resolved
- A list of ambiguities or vague points in the conversation

Here's a proposed sample JSON structure:

```json
{
  "summary": {
    "customerIssue": "Brief description of the main problem or reason for the call",
    "resolution": "How the issue was addressed or resolved, if applicable",
    "followUpRequired": true/false,
    "followUpDetails": "Description of any necessary follow-up actions, or null if none required"
  },
  "status": "COMPLETE",
  "ambiguities": ["List of any unclear or vague points in the conversation, or an empty array if none"]
}
```

Let's create a new piece of our prompt that includes specific instructions, including:
- Create a summary focusing on the main issue, resolution, and any follow-up actions required.
- Generate a JSON output following our specific, standardized format.
- Omit specific customer information in the summaries.
- Keep each piece of the summary short.

Here's an attempt at providing the output instructions, including our specific output JSON format:

In [52]:
prompt_pt2 = """
Instructions:
1. Read the transcript carefully.
2. Analyze the transcript, focusing on the main issue, resolution, and any follow-up required.
3. Generate a JSON object summarizing the key aspects of the interaction according to the specified structure.

Important guidelines:
- Confidentiality: Omit all specific customer data like names, phone numbers, and email addresses.
- Character limit: Restrict each text field to a maximum of 100 characters.
- Maintain a professional tone in your summary.

Output format:
Generate a JSON object with the following structure:
<json>
{
  "summary": {
    "customerIssue": "Brief description of the main problem or reason for the call",
    "resolution": "How the issue was addressed or resolved, if applicable",
    "followUpRequired": true/false,
    "followUpDetails": "Description of any necessary follow-up actions, or null if none required"
  },
  "status": "COMPLETE",
  "ambiguities": ["List of any unclear or vague points in the conversation, or an empty array if none"]
}
</json>
"""

--- 

## Using XML tags and giving Claude room to think
Next, we'll employ two more prompting strategies: giving Claude room to think and using XML tags.
- We'll ask Claude to start by outputting `<thinking>` tags that contain its analysis.
- Then, we'll ask Claude to output its JSON output inside of `<json>`.

Here's the final piece of our first draft prompt:

In [53]:
prompt_pt3 = """
Before generating the JSON, please analyze the transcript in <thinking> tags. 
Include your identification of the main issue, resolution, follow-up requirements, and any ambiguities. 
Then, provide your JSON output in <json> tags.
"""


By asking Claude to put its analysis within `<thinking>` tags, we're prompting it to break down its thought process before formulating the final JSON output. This encourages a more thorough and structured approach to analyzing the transcript.
The `<thinking>` section allows us (and potentially other reviewers or systems) to see Claude's reasoning process. This transparency can be crucial for debugging and quality assurance purposes.


By separating the analysis (`<thinking`>) from the structured output (`<json>`), we create a clear distinction between Claude's interpretation of the transcript and its formatted summary. This can be helpful in cases where we might want to review the analysis separately from the JSON output, but also by isolating the JSON content inside of `<json>` tags, we make it easy to parse the final response and capture the JSON we want to work with.


--- 

## Testing our updated prompt

Here's the complete version of the prompt, constructed by combining the individual prompt pieces we've written so far:

In [56]:
system = """
You are an expert customer service analyst, skilled at extracting key information from call transcripts and summarizing them in a structured format.
Your task is to analyze customer service call transcripts and generate concise, accurate summaries while maintaining a professional tone.
"""

prompt = """
Analyze the following customer service call transcript and generate a JSON summary of the interaction:

<transcript>
[INSERT CALL TRANSCRIPT HERE]
</transcript>

Instructions:
1. Read the transcript carefully.
2. Analyze the transcript, focusing on the main issue, resolution, and any follow-up required.
3. Generate a JSON object summarizing the key aspects of the interaction according to the specified structure.

Important guidelines:
- Confidentiality: Omit all specific customer data like names, phone numbers, and email addresses.
- Character limit: Restrict each text field to a maximum of 100 characters.
- Maintain a professional tone in your summary.

Output format:
Generate a JSON object with the following structure:
<json>
{
  "summary": {
    "customerIssue": "Brief description of the main problem or reason for the call",
    "resolution": "How the issue was addressed or resolved, if applicable",
    "followUpRequired": true/false,
    "followUpDetails": "Description of any necessary follow-up actions, or null if none required"
  },
  "status": "COMPLETE",
  "ambiguities": ["List of any unclear or vague points in the conversation, or an empty array if none"]
}
</json>

Before generating the JSON, please analyze the transcript in <thinking> tags. 
Include your identification of the main issue, resolution, follow-up requirements, and any ambiguities. 
Then, provide your JSON output in <json> tags.
"""

Here's a function we can use to test our prompt:

In [57]:
def summarize_call_with_improved_prompt(transcript):
    final_prompt = prompt.replace("[INSERT CALL TRANSCRIPT HERE]", transcript)
    # Make the API call
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        system=system,
        max_tokens=4096,
        messages=[
            {"role": "user", "content": final_prompt}
        ]
    )
    print(response.content[0].text)

Let's test out the prompt using some of the call transcripts we previously defined:

In [58]:
summarize_call_with_improved_prompt(call1)

<thinking>
From the transcript, the main issue appears to be that the customer could not turn on their smart light bulb. The resolution provided by the agent was to reset the bulb by turning the power off for 5 seconds and then back on.

The agent did offer for the customer to call back if they needed further assistance, indicating potential follow-up may be required if the reset did not resolve the issue. However, no specific follow-up details were provided.

There do not seem to be any significant ambiguities in the conversation.
</thinking>

<json>
{
  "summary": {
    "customerIssue": "Unable to turn on smart light bulb",
    "resolution": "Agent instructed customer to reset the bulb by turning power off for 5 seconds, then back on",
    "followUpRequired": true,
    "followUpDetails": "Customer was advised to call back if the reset did not resolve the issue"
  },
  "status": "COMPLETE",
  "ambiguities": []
}
</json>


In [59]:
summarize_call_with_improved_prompt(call2)

<thinking>
Main issue: The customer's Acme SmartTherm thermostat is not maintaining the set temperature of 72°F, and the house is much warmer.

Resolution: The agent guided the customer through recalibrating the SmartTherm thermostat by:
1. Having the customer press and hold the menu button for 5 seconds.
2. Navigating to the "Calibration" menu and selecting it.
3. Adjusting the temperature to match the customer's room thermometer reading of 79°F.
4. Confirming the new calibration setting.

Follow-up required: Yes, the agent instructed the customer to check back in an hour to see if the recalibration resolved the temperature issue.

Ambiguities: None
</thinking>

<json>
{
  "summary": {
    "customerIssue": "Thermostat not maintaining set temperature, causing house to be much warmer.",
    "resolution": "Agent guided customer through recalibrating the thermostat to match room temperature.",
    "followUpRequired": true,
    "followUpDetails": "Customer to check back in an hour to see i

In [60]:
summarize_call_with_improved_prompt(call3)

<thinking>
Main issue: The customer's Acme SecureHome system alarm is going off randomly in the middle of the night, even though doors and windows are closed properly.

Resolution: The agent suggests running a diagnostic on the system to identify potential sensor malfunctions. The customer is transferred to the technical team to perform the diagnostic and resolve the issue.

Follow-up required: Yes, the technical team needs to follow up with the customer to diagnose and fix the alarm system problem.

Ambiguities: None identified in the conversation.
</thinking>

<json>
{
  "summary": {
    "customerIssue": "Customer's home security alarm system is going off randomly at night without apparent cause.",
    "resolution": "Agent suggests running a diagnostic to check for sensor malfunction and transfers customer to technical team.",
    "followUpRequired": true,
    "followUpDetails": "Technical team to diagnose and resolve the issue with the customer's alarm system."
  },
  "status": "COM

Those responses all look great! Let's try another call transcript that has a bit of ambiguity to it to see if the JSON result includes those ambiguities:

In [64]:
ambiguous_call = """
Agent: Thank you for calling Acme Smart Home Support. This is Alex. How may I assist you today?
Customer: Hi Alex, I'm having an issue with my SmartLock. It's not working properly.
Agent: I'm sorry to hear that. Can you tell me more about what's happening with your SmartLock?
Customer: Well, sometimes it doesn't lock when I leave the house. I think it might be related to my phone, but I'm not sure.
Agent: I see. When you say it doesn't lock, do you mean it doesn't respond to the auto-lock feature, or are you trying to lock it manually through the app?
Customer: Uh, both, I think. Sometimes one works, sometimes the other. It's inconsistent.
Agent: Okay. And you mentioned it might be related to your phone. Have you noticed any pattern, like it works better when you're closer to the door?
Customer: Maybe? I haven't really paid attention to that.
Agent: Alright. Let's try to troubleshoot this. First, can you tell me what model of SmartLock you have?
Customer: I'm not sure. I bought it about six months ago, if that helps.
Agent: That's okay. Can you see a model number on the lock itself?
Customer: I'd have to go check. Can we just assume it's the latest model?
Agent: Well, knowing the exact model would help us troubleshoot more effectively. But let's continue with what we know. Have you tried resetting the lock recently?
Customer: I think so. Or maybe that was my SmartTherm. I've been having issues with that too.
Agent: I see. It sounds like we might need to do a full diagnostic on your SmartLock. Would you be comfortable if I walked you through that process now?
Customer: Actually, I have to run to an appointment. Can I call back later?
Agent: Of course. Before you go, is there a good contact number where our technical team can reach you for a more in-depth troubleshooting session?
Customer: Sure, you can reach me at 555... oh wait, that's my old number. Let me check my new one... You know what, I'll just call back when I have more time.
Agent: I understand. We're here 24/7 when you're ready to troubleshoot. Is there anything else I can help with before you go?
Customer: No, that's it. Thanks.
Agent: You're welcome. Thank you for choosing Acme Smart Home. Have a great day!
"""

summarize_call_with_improved_prompt(ambiguous_call)

<thinking>
Main Issue: The customer is experiencing issues with their Acme SmartLock not consistently locking automatically or manually through the app.

Resolution: The agent attempted to troubleshoot by asking for the specific SmartLock model and suggesting a reset, but the customer had to leave before completing the troubleshooting process.

Follow-Up Required: Yes, the customer needs to call back to complete a full diagnostic and troubleshooting session with the technical team.

Ambiguities:
- The customer was unsure if the issue was related to their phone or not, suggesting a potential connectivity problem.
- The customer mentioned having issues with another Acme product (SmartTherm), but it's unclear if those issues are related to the SmartLock problem.
- The customer's contact number was not clearly provided, which could make follow-up more difficult.
</thinking>

<json>
{
  "summary": {
    "customerIssue": "SmartLock not consistently locking automatically or manually through t

Great! Everything seems to be working as intended

--- 

## Edge cases

So far, all of the call transcripts we've tried have been relatively straightforward customer service calls. In the real world, we would expect to also encounter transcripts that perhaps we don't want to summarize, including: 

- Calls with connection issues
- Calls with language barriers
- Calls with garbled transcripts
- Calls with irrational or upset customers

Remember, our goal is to summarize these calls to help gauge the effectiveness of the customer service we offer.  If we include these edge-case calls in the summaries, we'll likely get skewed results.

Let's see what happens with some of these edge cases with our current prompt.  Below we've defined some new call transcripts:

In [65]:
wrong_number_call = """
Agent: Acme Smart Home Support, Lisa speaking. How can I help you?
Customer: Is this tech support?
Agent: Yes, this is technical support for Acme Smart Home devices. What can I help you with?
Customer: Sorry, wrong number.
Agent: No problem. Have a nice day.
"""

incomplete_call = """
Agent: Acme Smart Home Support, this is Sarah. How can I assist you today?
Customer: The thing isn't working.
Agent: I'm sorry to hear that. Could you please specify which device you're having trouble with?
Customer: You know, the usual one. Gotta go, bye.
Agent: Wait, I need more infor... [call disconnected]
"""

garbled_call = """
Agent: Thank you for calling Acme Smart Home Support. This is Alex. How may I assist you today?
Customer: [garbled voice]
Agent: Hello? Are you there?
"""

language_barrier_call = """
Agent: Acme Smart Home Support, Sarah speaking. How can I help you today?
Customer: [Speaking in Spanish]
Agent: I apologize, but I don't speak Spanish. Do you speak English?
Customer: [Continues Spanish]
Agent: One moment please, I'll try to get a translator on the line...
"""


Let's run these edge-case transcripts through our prompt and see what sort of results we get:

In [66]:
summarize_call_with_improved_prompt(wrong_number_call)

<thinking>
Issue: The customer appears to have dialed the wrong number for technical support.
Resolution: Since it was a wrong number, there was no issue to resolve. The agent politely concluded the call.
Follow-up: No follow-up is required since it was a misdialed call.
Ambiguities: There are no apparent ambiguities in this brief conversation.
</thinking>

<json>
{
  "summary": {
    "customerIssue": "The customer dialed the wrong number for technical support",
    "resolution": "The agent concluded the call politely since it was a misdialed number",
    "followUpRequired": false,
    "followUpDetails": null
  },
  "status": "COMPLETE",
  "ambiguities": []
}
</json>


In [67]:
summarize_call_with_improved_prompt(incomplete_call)

<thinking>
In this transcript, the main issue is unclear as the customer does not provide sufficient details about the device they are having trouble with. The agent attempts to clarify the issue, but the customer disconnects the call before providing more information.

There is no resolution since the issue is not clearly identified. However, follow-up is required to gather more details from the customer about the specific device and the problem they are experiencing.

The key ambiguity is the lack of clarity about the device and the nature of the problem. The customer's vague statements ("the thing isn't working" and "the usual one") do not provide enough information for the agent to diagnose or resolve the issue.
</thinking>

<json>
{
  "summary": {
    "customerIssue": "Customer reported an unspecified device was not working but did not provide further details.",
    "resolution": "No resolution was possible due to lack of information from the customer.",
    "followUpRequired": tr

In [68]:
summarize_call_with_improved_prompt(garbled_call)

<thinking>
From the transcript, it appears the customer called Acme Smart Home Support, but their voice was garbled, and the agent could not understand them. With this limited information, it is unclear what the main issue or reason for the call was. There was no resolution provided, and it is ambiguous whether follow-up is required since the issue itself is unknown. The key ambiguity is the lack of clear communication from the customer, preventing the agent from understanding the problem.
</thinking>

<json>
{
  "summary": {
    "customerIssue": "Unclear due to garbled voice from the customer",
    "resolution": "No resolution provided since the issue could not be understood",
    "followUpRequired": true,
    "followUpDetails": "Agent should try to reconnect with the customer for clearer communication"
  },
  "status": "COMPLETE",
  "ambiguities": ["The customer's voice was garbled, preventing understanding of the issue"]
}
</json>


In [69]:
summarize_call_with_improved_prompt(language_barrier_call)

<thinking>
Main issue: The customer called and spoke in Spanish, but the agent could not understand Spanish.
Resolution: The agent tried to get a translator on the line to resolve the language barrier.
Follow-up required: Yes, the agent needs to connect with a Spanish translator to assist the customer.
Ambiguities: It is unclear why the customer called, as the reason for their call is not stated in the transcript.
</thinking>

<json>
{
  "summary": {
    "customerIssue": "Customer spoke in a language the agent did not understand (Spanish).",
    "resolution": "Agent attempted to get a translator to resolve the language barrier.",
    "followUpRequired": true,
    "followUpDetails": "Agent needs to connect the customer with a Spanish translator."
  },
  "status": "COMPLETE",
  "ambiguities": ["Reason for the customer's call is not stated in the transcript."]
}
</json>


Unfortunately, we're getting full summaries for these edge-case transcripts.  Here are some problematic parts of the responses: 

>  "customerIssue": "Customer spoke in a language the agent did not understand (Spanish)."

> "customerIssue": "Unclear due to garbled voice from the customer"

> "customerIssue": "The customer dialed the wrong number for technical support" 

Remember that our goal is to summarize our customer service calls to get some insight into how effective our customer service team is.  These edge-case transcripts are resulting in complete summaries that will cause problems when analyzing all the summaries.  We'll need to decide on a strategy for handling these calls.

--- 

## Further prompt improvements

As we previously saw, our prompt is currently generating full summaries for edge-case transcripts.  We want to change this behavior.  We have a couple of options for how we handle these edge-cases:

- Flag them in some way to indicate they are not summarizable, allowing for later human-review.
- Categorize them separately (e.g., "technical difficulty,"  "language barrier,"  etc.).

For simplicity's sake, we'll opt to flag these edge-case calls by asking the model to output JSON that looks like this: 

```json
{
  "status": "INSUFFICIENT_DATA"
}
```

In order to make this work, we'll need to update our prompt in the following ways:
- Add instructions explaining the desired "INSUFFICIENT_DATA" output
- Add examples to show summarizable and non-summarizable transcripts along with their corresponding JSON outputs.


### Updating our instructions

Let's write a new part of the instructions portion of the prompt to explain when the model should output our "INSUFFICIENT_DATA" JSON.

In [70]:
# Just the new content.  We'll look at the entire prompt in a moment
new_instructions_addition = """
Insufficient data criteria:
   If either of these conditions are met:
   a) The transcript has fewer than 5 total exchanges, or
   b) The customer's issue is unclear
   c) The call is garbled, incomplete, or is hindered by a language barrier
   Then return ONLY the following JSON:
   {
     "status": "INSUFFICIENT_DATA"
   }
"""

### Adding examples

As we discussed previously in this course, it's almost always a good idea to add examples to a prompt.  In this specific use case, examples will help Claude generally understand the types of summaries we want for both summarizable and non-summarizable call transcripts.

Here's a set of examples we could include in our prompt:

In [71]:
examples_for_prompt = """
<examples>
1. Complete interaction:
<transcript>
Agent: Thank you for calling Acme Smart Home Support. This is Alex. How may I assist you today?
Customer: Hi Alex, my Acme SmartTherm isn't maintaining the temperature I set. It's set to 72 but the house is much warmer.
Agent: I'm sorry to hear that. Let's troubleshoot. Is your SmartTherm connected to Wi-Fi?
Customer: Yes, the Wi-Fi symbol is showing on the display.
Agent: Great. Let's recalibrate your SmartTherm. Press and hold the menu button for 5 seconds.
Customer: Okay, done. A new menu came up.
Agent: Perfect. Navigate to "Calibration" and press select. Adjust the temperature to match your room thermometer.
Customer: Alright, I've set it to 79 degrees to match.
Agent: Great. Press select to confirm. It will recalibrate, which may take a few minutes. Check back in an hour to see if it's fixed.
Customer: Okay, I'll do that. Thank you for your help, Alex.
Agent: You're welcome! Is there anything else I can assist you with today?
Customer: No, that's all. Thanks again.
Agent: Thank you for choosing Acme Smart Home. Have a great day!
</transcript>

<thinking>
Main issue: SmartTherm not maintaining set temperature
Resolution: Guided customer through recalibration process
Follow-up: Not required, but customer should check effectiveness after an hour
Ambiguities: None identified
</thinking>

<json>
{
  "summary": {
    "customerIssue": "SmartTherm not maintaining set temperature, showing higher than set 72 degrees",
    "resolution": "Guided customer through SmartTherm recalibration process",
    "followUpRequired": false,
    "followUpDetails": null
  },
  "status": "COMPLETE",
  "ambiguities": []
}
</json>

2. Interaction requiring follow-up:
<transcript>
Agent: Acme Smart Home Support, this is Jamie. How can I help you?
Customer: Hi, I just installed my new Acme SmartCam, but I can't get it to connect to my Wi-Fi.
Agent: I'd be happy to help. Are you using the Acme Smart Home app?
Customer: Yes, I have the app on my phone.
Agent: Great. Make sure your phone is connected to the 2.4GHz Wi-Fi network, not the 5GHz one.
Customer: Oh, I'm on the 5GHz network. Should I switch?
Agent: Yes, please switch to the 2.4GHz network. The SmartCam only works with 2.4GHz.
Customer: Okay, done. Now what?
Agent: Open the app, select 'Add Device', choose 'SmartCam', and follow the on-screen instructions.
Customer: It's asking for a password now.
Agent: Enter your Wi-Fi password and it should connect.
Customer: It's still not working. I keep getting an error message.
Agent: I see. In that case, I'd like to escalate this to our technical team. They'll contact you within 24 hours.
Customer: Okay, that sounds good. Thank you for trying to help.
Agent: You're welcome. Is there anything else you need assistance with?
Customer: No, that's all for now. Thanks again.
Agent: Thank you for choosing Acme Smart Home. Have a great day!
</transcript>

<thinking>
Main issue: Customer unable to connect new SmartCam to Wi-Fi
Resolution: Initial troubleshooting unsuccessful, issue escalated to technical team
Follow-up: Required, technical team to contact customer within 24 hours
Ambiguities: Specific error message customer is receiving not mentioned
</thinking>

<json>
{
  "summary": {
    "customerIssue": "Unable to connect new SmartCam to Wi-Fi",
    "resolution": "Initial troubleshooting unsuccessful, issue escalated to technical team",
    "followUpRequired": true,
    "followUpDetails": "Technical team to contact customer within 24 hours for further assistance"
  },
  "status": "COMPLETE",
  "ambiguities": ["Specific error message customer is receiving not mentioned"]
}
</json>

3. Insufficient data:
<transcript>
Agent: Acme Smart Home Support, this is Sam. How may I assist you?
Customer: Hi, my smart lock isn't working.
Agent: I'm sorry to hear that. Can you tell me more about the issue?
Customer: It just doesn't work. I don't know what else to say.
Agent: Okay, when did you first notice the problem? And what model of Acme smart lock do you have?
Customer: I don't remember. Listen, I have to go. I'll call back later.
Agent: Alright, we're here 24/7 if you need further assistance. Have a good day.
</transcript>

<thinking>
This transcript has fewer than 5 exchanges and the customer's issue is unclear. The customer doesn't provide specific details about the problem with the smart lock or respond to the agent's questions. This interaction doesn't provide sufficient information for a complete summary.
</thinking>

<json>
{
  "status": "INSUFFICIENT_DATA"
}
</json>
</examples>
"""

Note that the examples cover three different situations: 
* A complete interaction that does not require follow up
* A complete interaction that does require follow up and contains ambiguities
* A non-summarizable interaction that contains insufficient data

When providing examples to Claude, it's important to cover a variety of input/output pairs.

--- 

## Our final prompt

Let's combine our initial prompt with the additions we made in the previous section:
* the instructions on handling calls with insufficient data
* the set of example inputs and outputs

This is the new complete prompt:

In [75]:
system = """
You are an expert customer service analyst, skilled at extracting key information from call transcripts and summarizing them in a structured format.
Your task is to analyze customer service call transcripts and generate concise, accurate summaries while maintaining a professional tone.
"""

prompt = """
Analyze the following customer service call transcript and generate a JSON summary of the interaction:

<transcript>
[INSERT CALL TRANSCRIPT HERE]
</transcript>

Instructions:
<instructions>
1. Read the transcript carefully.
2. Analyze the transcript, focusing on the main issue, resolution, and any follow-up required.
3. Generate a JSON object summarizing the key aspects of the interaction according to the specified structure.

Important guidelines:
- Confidentiality: Omit all specific customer data like names, phone numbers, and email addresses.
- Character limit: Restrict each text field to a maximum of 100 characters.
- Maintain a professional tone in your summary.

Output format:
Generate a JSON object with the following structure:
<json>
{
  "summary": {
    "customerIssue": "Brief description of the main problem or reason for the call",
    "resolution": "How the issue was addressed or resolved, if applicable",
    "followUpRequired": true/false,
    "followUpDetails": "Description of any necessary follow-up actions, or null if none required"
  },
  "status": "COMPLETE",
  "ambiguities": ["List of any unclear or vague points in the conversation, or an empty array if none"]
}
</json>

Insufficient data criteria:
   If any of these conditions are met:
   a) The transcript has fewer than 5 total exchanges
   b) The customer's issue is unclear
   c) The call is garbled, incomplete, or is hindered by a language barrier
   Then return ONLY the following JSON:
   {
     "status": "INSUFFICIENT_DATA"
   }

Examples: 
<examples>
1. Complete interaction:
<transcript>
Agent: Thank you for calling Acme Smart Home Support. This is Alex. How may I assist you today?
Customer: Hi Alex, my Acme SmartTherm isn't maintaining the temperature I set. It's set to 72 but the house is much warmer.
Agent: I'm sorry to hear that. Let's troubleshoot. Is your SmartTherm connected to Wi-Fi?
Customer: Yes, the Wi-Fi symbol is showing on the display.
Agent: Great. Let's recalibrate your SmartTherm. Press and hold the menu button for 5 seconds.
Customer: Okay, done. A new menu came up.
Agent: Perfect. Navigate to "Calibration" and press select. Adjust the temperature to match your room thermometer.
Customer: Alright, I've set it to 79 degrees to match.
Agent: Great. Press select to confirm. It will recalibrate, which may take a few minutes. Check back in an hour to see if it's fixed.
Customer: Okay, I'll do that. Thank you for your help, Alex.
Agent: You're welcome! Is there anything else I can assist you with today?
Customer: No, that's all. Thanks again.
Agent: Thank you for choosing Acme Smart Home. Have a great day!
</transcript>

<thinking>
Main issue: SmartTherm not maintaining set temperature
Resolution: Guided customer through recalibration process
Follow-up: Not required, but customer should check effectiveness after an hour
Ambiguities: None identified
</thinking>

<json>
{
  "summary": {
    "customerIssue": "SmartTherm not maintaining set temperature, showing higher than set 72 degrees",
    "resolution": "Guided customer through SmartTherm recalibration process",
    "followUpRequired": false,
    "followUpDetails": null
  },
  "status": "COMPLETE",
  "ambiguities": []
}
</json>

2. Interaction requiring follow-up:
<transcript>
Agent: Acme Smart Home Support, this is Jamie. How can I help you?
Customer: Hi, I just installed my new Acme SmartCam, but I can't get it to connect to my Wi-Fi.
Agent: I'd be happy to help. Are you using the Acme Smart Home app?
Customer: Yes, I have the app on my phone.
Agent: Great. Make sure your phone is connected to the 2.4GHz Wi-Fi network, not the 5GHz one.
Customer: Oh, I'm on the 5GHz network. Should I switch?
Agent: Yes, please switch to the 2.4GHz network. The SmartCam only works with 2.4GHz.
Customer: Okay, done. Now what?
Agent: Open the app, select 'Add Device', choose 'SmartCam', and follow the on-screen instructions.
Customer: It's asking for a password now.
Agent: Enter your Wi-Fi password and it should connect.
Customer: It's still not working. I keep getting an error message.
Agent: I see. In that case, I'd like to escalate this to our technical team. They'll contact you within 24 hours.
Customer: Okay, that sounds good. Thank you for trying to help.
Agent: You're welcome. Is there anything else you need assistance with?
Customer: No, that's all for now. Thanks again.
Agent: Thank you for choosing Acme Smart Home. Have a great day!
</transcript>

<thinking>
Main issue: Customer unable to connect new SmartCam to Wi-Fi
Resolution: Initial troubleshooting unsuccessful, issue escalated to technical team
Follow-up: Required, technical team to contact customer within 24 hours
Ambiguities: Specific error message customer is receiving not mentioned
</thinking>

<json>
{
  "summary": {
    "customerIssue": "Unable to connect new SmartCam to Wi-Fi",
    "resolution": "Initial troubleshooting unsuccessful, issue escalated to technical team",
    "followUpRequired": true,
    "followUpDetails": "Technical team to contact customer within 24 hours for further assistance"
  },
  "status": "COMPLETE",
  "ambiguities": ["Specific error message customer is receiving not mentioned"]
}
</json>

3. Insufficient data:
<transcript>
Agent: Acme Smart Home Support, this is Sam. How may I assist you?
Customer: Hi, my smart lock isn't working.
Agent: I'm sorry to hear that. Can you tell me more about the issue?
Customer: It just doesn't work. I don't know what else to say.
Agent: Okay, when did you first notice the problem? And what model of Acme smart lock do you have?
Customer: I don't remember. Listen, I have to go. I'll call back later.
Agent: Alright, we're here 24/7 if you need further assistance. Have a good day.
</transcript>

<thinking>
This transcript has fewer than 5 exchanges and the customer's issue is unclear. The customer doesn't provide specific details about the problem with the smart lock or respond to the agent's questions. This interaction doesn't provide sufficient information for a complete summary.
</thinking>

<json>
{
  "status": "INSUFFICIENT_DATA"
}
</json>
</examples>
</instructions>

Before generating the JSON, please analyze the transcript in <thinking> tags. 
Include your identification of the main issue, resolution, follow-up requirements, and any ambiguities. 
Then, provide your JSON output in <json> tags.
"""

The above prompt is quite long, but here is the general structure:
- The system prompt sets the context, role, and tone for the model.
- The main prompt includes the following:
    - the call transcript
    - a set of instructions containing:
        - general instructions
        - guidelines 
        - output format requirements
        - details on handling edge-case calls
        - examples
    - details on the XML tags to use in the output

Here's a summary to help visualize the flow of the prompt: 

```txt
Analyze the following customer service call transcript and generate a JSON summary of the interaction:

<transcript>
[INSERT CALL TRANSCRIPT HERE]
</transcript>

<instructions>
- General instructions and guidelines
- Output JSON format description
- Insufficient data (edge-case) criteria
<examples>
varied example inputs and outputs
</examples>
</instructions>

Before generating the JSON, please analyze the transcript in <thinking> tags. 
Include your identification of the main issue, resolution, follow-up requirements, and any ambiguities. 
Then, provide your JSON output in <json> tags.

```

Let's test the final prompt with a new function.  Note that this function extracts the JSON summary content inside the `<json>` tags:

In [76]:
import re

def summarize_call_with_final_prompt(transcript):
    final_prompt = prompt.replace("[INSERT CALL TRANSCRIPT HERE]", transcript)
    # Make the API call
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        system=system,
        max_tokens=4096,
        messages=[
            {"role": "user", "content": final_prompt}
        ]
    )
    
    # Extract content between <json> tags
    json_content = re.search(r'<json>(.*?)</json>', response.content[0].text, re.DOTALL)
    
    if json_content:
        print(json_content.group(1).strip())
    else:
        print("No JSON content found in the response.")

Let's test it out with a bunch of our existing call variables:

In [77]:
summarize_call_with_final_prompt(call1)

{
  "summary": {
    "customerIssue": "Unable to turn on smart light bulb",
    "resolution": "Agent guided customer to reset the bulb by cycling power off and on",
    "followUpRequired": false,
    "followUpDetails": null
  },
  "status": "COMPLETE",
  "ambiguities": []
}


In [78]:
summarize_call_with_final_prompt(call3)

{
  "summary": {
    "customerIssue": "Acme SecureHome alarm system going off randomly multiple times at night without apparent cause",
    "resolution": "Initial troubleshooting steps taken, but issue unresolved. Customer transferred to technical team for diagnostics",
    "followUpRequired": true,
    "followUpDetails": "Technical team to diagnose and resolve issue with alarm system"
  },
  "status": "COMPLETE",
  "ambiguities": []
}


Let's try our call transcript that should result in a summary with a non-empty `ambiguities` array:

In [79]:
summarize_call_with_final_prompt(ambiguous_call)

{
  "summary": {
    "customerIssue": "SmartLock not reliably locking automatically or through app, behavior is inconsistent",
    "resolution": "Troubleshooting attempted but incomplete due to lack of model details, customer had to leave",
    "followUpRequired": true,
    "followUpDetails": "Customer to call back for further troubleshooting of SmartLock issue when available"
  },
  "status": "COMPLETE",
  "ambiguities": [
    "Unclear if related SmartTherm issue mentioned",
    "SmartLock model not identified",
    "Customer's contact number not confirmed"
  ]
}


Now let's try some of our edge case prompts that we do not want summarized:

In [80]:
summarize_call_with_final_prompt(garbled_call)

{
  "status": "INSUFFICIENT_DATA"
}


In [82]:
summarize_call_with_final_prompt(language_barrier_call)

{
  "status": "INSUFFICIENT_DATA"
}


In [83]:
summarize_call_with_final_prompt(incomplete_call)

{
  "status": "INSUFFICIENT_DATA"
}


Great! We're getting the exact outputs we want! Let's try pushing it even further:

In [84]:
summarize_call_with_final_prompt("blah blah blah")

{
  "status": "INSUFFICIENT_DATA"
}


In [85]:
summarize_call_with_final_prompt("")

{
  "status": "INSUFFICIENT_DATA"
}


Excellent, the prompt is handling all of our edge cases!

---

## Wrap up

In this lesson, we walked through the process of developing a complex prompt for summarizing customer service call transcripts. Let's recap the prompting techniques we employed:

* System Prompt: We used a system prompt to set the overall context and role for Claude.
* Structured Input: We placed the call transcript at the beginning of the prompt using XML tags.
* Clear Instructions: We provided detailed guidelines on what to focus on and how to structure the output.
* Output Formatting: We specified a JSON structure for the summary, ensuring consistent and easily parseable results.
* Handling Edge Cases: We added criteria for identifying calls with insufficient data.
* Examples: We included diverse examples to illustrate desired outputs for different scenarios.
* Thinking Aloud: We asked Claude to show its analysis in <thinking> tags before providing the final JSON output.


By employing these techniques, we created a robust prompt capable of generating structured summaries for a wide range of customer service call transcripts, while appropriately handling edge cases. This approach can be adapted to many other complex prompting scenarios beyond call summarization.


**Important Note:** While we've developed a sophisticated prompt that appears to handle our test cases well, it's crucial to understand that this prompt is not yet production-ready. What we've created is a promising starting point, but it requires extensive testing and evaluation before it can be reliably used in a real-world setting. Our current eye-ball test evaluation has been based on a small set of examples. This is not representative of the diverse and often unpredictable nature of real customer service calls. To ensure the prompt's effectiveness and reliability, we need to implement a comprehensive evaluation process that includes quantitative metrics.  Robust, data-driven evaluations are the key to bridging the gap between a promising prototype and a reliable, production-grade solution.